In [ ]:
import os
import numpy as np
import pandas as pd

mock_path = '/path/to/your/mock/files'  # Directory path where data files are located
column_list = ['Gaze fixation duration', 'AU01', 'AU02', 'AU04', 'AU05', 'AU06', 'AU07', 'AU09', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'AU20', 'AU23', 'AU25', 'AU26', 'AU28', 'non_AU45', 'Neutral']
index_list = list(range(1, 301))

mock_output_path = '/path/to/your/output/directory/EC_control'  # Directory path to store result files
mock_analysis_output_path = '/path/to/your/analysis/output/directory'  # Directory path to store analysis result files

# Set thresholds for detecting gaze movement
x_threshold = 0.005
y_threshold = 0.005
z_threshold = 0.001

T_total_result = np.zeros((144, 300, 20))  # 144: Number of Truth, 300: Number of frames, 20: Number of EC control metrics
F_total_result = np.zeros((72, 300, 20))  # 72: Number of Deception
f_idx = 0
t_idx = 0

for f in sorted(os.listdir(mock_path)):
    df = pd.read_csv(os.path.join(mock_path, f))
    Dataframe = df.iloc[:, 696:714]
    Dataframe = Dataframe.to_numpy()
    result = np.zeros((300, 20))  # 300: Number of frames, 20: Number of EC control metrics
    
    ### Gaze ###
    for frame_len in range(2, 300):
        # Extract gaze vectors for left and right eyes
        gaze_vectors = df.iloc[0:frame_len, 5:11]
        gaze_vectors = gaze_vectors.to_numpy()
        
        # Extract blink data
        blink = df.iloc[0:frame_len, -1]
        blink = blink.to_numpy()

        # Determine significant eye movements for left and right eyes
        for i in range(1, frame_len):
            # Initialize maximum still time when no movement is detected
            max_still_time = 0
            still_time = 0

            # Extract x, y, z coordinates of the left and right eyes
            x_right, y_right, z_right, x_left, y_left, z_left = gaze_vectors[i, :]

            # Only consider frames when eyes are not blinking
            if float(blink[i]) == 0:
                # Determine significant eye movements for left and right eyes
                if round(abs(x_right - gaze_vectors[i - 1][0]), 3) > x_threshold or \
                   round(abs(y_right - gaze_vectors[i - 1][1]), 3) > y_threshold or \
                   round(abs(z_right - gaze_vectors[i - 1][2]), 3) > z_threshold or \
                   round(abs(x_left - gaze_vectors[i - 1][3]), 3) > x_threshold or \
                   round(abs(y_left - gaze_vectors[i - 1][4]), 3) > y_threshold or \
                   round(abs(z_left - gaze_vectors[i - 1][5]), 3) > z_threshold:
                    still_time = 0  # Reset time when movement is detected
                else:
                    still_time += 1  # Increase time when no movement is detected

                # Update maximum still time
                if still_time > max_still_time:
                    max_still_time = still_time
        result[frame_len, 0] = max_still_time

    ### AU Duration ###
    for AU in range(17):
        for frame_len in range(2, 300):
            count_ones = 0
            EC_count = 0
            for i in range(frame_len):
                if float(Dataframe[i, AU]) == 1:
                    count_ones += 1
                    EC_count = max(EC_count, count_ones)
                else:
                    count_ones = 0
            result[frame_len, AU + 1] = EC_count

    ### Duration without Eye Blinking ###
    for frame_len in range(2, 300):
        count_zeros = 0
        EC_count = 0
        for i in range(frame_len):
            if float(Dataframe[i, -1]) == 0:
                count_zeros += 1
                EC_count = max(EC_count, count_zeros)
            else:
                count_zeros = 0
        result[frame_len, 18] = EC_count

    ### Duration of Neutral Expression ###
    for frame_len in range(2, 300):
        count_zeros = 0
        EC_count = 0
        for i in range(frame_len):
            if np.all(Dataframe[i, :] == 0):
                count_zeros += 1
                EC_count = max(EC_count, count_zeros)
            else:
                count_zeros = 0
        result[frame_len, 19] = EC_count

    EC_count_df = pd.DataFrame(result, columns=column_list, index=index_list)
    if not os.path.exists(mock_output_path):
        os.makedirs(mock_output_path)
    EC_count_df.to_csv(os.path.join(mock_output_path, f.split('.')[0] + '.csv'))

    if f.split('.')[0].split('_')[-1] == 'T':
        T_total_result[t_idx, :, :] = result
        t_idx += 1
    elif f.split('.')[0].split('_')[-1] == 'F':
        F_total_result[f_idx, :, :] = result
        f_idx += 1

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind_from_stats

column_list = ['Gaze fixation duration', 'AU01', 'AU02', 'AU04', 'AU05', 'AU06', 'AU07', 'AU09', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'AU20', 'AU23', 'AU25', 'AU26', 'AU28', 'non_AU45', 'Neutral']
index_list = list(range(1, 301))

# Dictionary to store necessary statistics
statistics = {}
confidence_interval = 0.95  # Confidence interval (95%)

EC_control_stat_result_pvalue = np.zeros((300, 20))  # 300: Number of frames, 20: Number of EC control metrics
EC_control_stat_result_tstat = np.zeros((300, 20))

for AU in range(20):
    for frame in range(300):
        observation_A = T_total_result[:, frame, AU]
        observation_B = F_total_result[:, frame, AU]
        observation_statistics = {}

        # T-test between Group A and Group B
        truth_mean = np.mean(T_total_result[:, frame, AU])
        truth_std = np.std(T_total_result[:, frame, AU])
        truth_sample_size = T_total_result.shape[0]

        deception_mean = np.mean(F_total_result[:, frame, AU])
        deception_std = np.std(F_total_result[:, frame, AU])
        deception_sample_size = F_total_result.shape[0]

        t_statistic, p_value = ttest_ind_from_stats(
            truth_mean, truth_std, truth_sample_size,
            deception_mean, deception_std, deception_sample_size
        )

        EC_control_stat_result_pvalue[frame, AU] = p_value
        EC_control_stat_result_tstat[frame, AU] = t_statistic

df1 = pd.DataFrame(EC_control_stat_result_pvalue, columns=column_list, index=index_list)
df1.to_csv(os.path.join(mock_analysis_output_path, 'mock_EC_control_stat_result_pvalue.csv'), index=True)
df2 = pd.DataFrame(EC_control_stat_result_tstat, columns=column_list, index=index_list)
df2.to_csv(os.path.join(mock_analysis_output_path, 'mock_EC_control_stat_result_tstat.csv'), index=True)